In [1]:
bucket = spark._jsc.hadoopConfiguration().get("fs.gs.system.bucket")
url = "gs://" + bucket + "/data/chicago-taxi-trip/cleaned-chicago-taxi/"

In [2]:
from pyspark.sql.types import (StructType, 
                               StructField, 
                               DoubleType,
                               IntegerType,
                               StringType,
                               TimestampType)

taxi_schema = StructType([StructField('unique_key', StringType(), True),
                StructField('taxi_id', StringType(), True),
                StructField('trip_start_timestamp', TimestampType(), True), 
                StructField('trip_end_timestamp', TimestampType(), True), 
                StructField('trip_seconds', IntegerType(), True), 
                StructField('trip_miles', IntegerType(), True), 
                StructField('pickup_census_tract', StringType(), True), 
                StructField('dropoff_census_tract', StringType(), True), 
                StructField('pickup_community_area', IntegerType(), True), 
                StructField('dropoff_community_area', IntegerType(), True), 
                StructField('fare', IntegerType(), True), 
                StructField('tips', IntegerType(), True), 
                StructField('tolls', IntegerType(), True), 
                StructField('extras', IntegerType(), True), 
                StructField('trip_total', IntegerType(), True), 
                StructField('payment_type', StringType(), True), 
                StructField('company', StringType(), True), 
                StructField('pickup_latitude', StringType(), True), 
                StructField('pickup_longitude', StringType(), True), 
                StructField('pickup_location', StringType(), True), 
                StructField('dropoff_latitude', StringType(), True), 
                StructField('dropoff_longitude', StringType(), True), 
                StructField('dropoff_location', StringType(), True)])

taxi = spark.read.format("csv").option("header", "true").schema(taxi_schema).csv(url)

In [13]:
taxi.cache()

DataFrame[unique_key: string, taxi_id: string, trip_start_timestamp: timestamp, trip_end_timestamp: timestamp, trip_seconds: int, trip_miles: int, pickup_census_tract: string, dropoff_census_tract: string, pickup_community_area: int, dropoff_community_area: int, fare: int, tips: int, tolls: int, extras: int, trip_total: int, payment_type: string, company: string, pickup_latitude: string, pickup_longitude: string, pickup_location: string, dropoff_latitude: string, dropoff_longitude: string, dropoff_location: string]

In [3]:
taxi.show(5)

+--------------------+--------------------+--------------------+-------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+----+-----+------+----------+-------------+--------------------+---------------+----------------+--------------------+----------------+-----------------+----------------+
|          unique_key|             taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles|pickup_census_tract|dropoff_census_tract|pickup_community_area|dropoff_community_area|fare|tips|tolls|extras|trip_total| payment_type|             company|pickup_latitude|pickup_longitude|     pickup_location|dropoff_latitude|dropoff_longitude|dropoff_location|
+--------------------+--------------------+--------------------+-------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+----+-----+------+----------+-------------+------------------

In [4]:
taxi.describe().show()

+-------+--------------------+--------------------+------------------+------------------+-------------------+--------------------+---------------------+----------------------+-----------------+------------------+-----+------+----------+-------------------+--------------------+--------------------+-------------------+--------------------+------------------+-----------------+----------------+
|summary|          unique_key|             taxi_id|      trip_seconds|        trip_miles|pickup_census_tract|dropoff_census_tract|pickup_community_area|dropoff_community_area|             fare|              tips|tolls|extras|trip_total|       payment_type|             company|     pickup_latitude|   pickup_longitude|     pickup_location|  dropoff_latitude|dropoff_longitude|dropoff_location|
+-------+--------------------+--------------------+------------------+------------------+-------------------+--------------------+---------------------+----------------------+-----------------+------------------+

In [5]:
taxi.describe().toPandas()

,summary,unique_key,taxi_id,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,count,20352161,20352161,2829328,20352161,20352161,5214529,15914800,16794907,19161717,...,0,0,17827660,17827660,17333531,17333531,17333531,20352161,0,0
1,mean,None,None,1.9476380963960347,26.07434694527033,21.03200981949779,17.394083147298634,0.9409646995249704,0.004890113413548524,1.220027307573742,...,None,None,-87.6724718774177,None,41.88627384287559,-87.64957851664845,None,17.208450051078113,None,None
2,stddev,None,None,5.370967954923644,23.637782212640044,19.07993369795151,24.615003101815958,2.176381276148622,0.4702432127410075,5.496590806089562,...,None,None,0.09270527304749865,None,0.04935942645499254,0.05718929883108902,None,26.80213265681773,None,None
3,min,000001bde866596a38db452f082a0ed75b4829b1,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,0,-1,-1,0,0,0,0,...,None,None,-87.530712484,POINT (-87.5307124836 41.7030053028),41.650221676,-87.531386257,POINT (-87.5313862567 41.7204632831),1,None,None
4,max,ffffff1aae5322736637e16dd2faecb5dfebe81a,fff84aa08ac78890c6e7da64b817cbd9aad6a124104e09...,950,77,9,99,500,1000,9555,...,None,None,-87.913624596,POINT (-87.913624596 41.9802643146),42.021223593,-87.913624596,POINT (-87.913624596 41.9802643146),999,None,None


In [12]:
from pyspark.sql.functions import col, sum
null_counts = taxi.select([sum(col(column).isNull().cast("int")).alias(column) for column in taxi.columns])

+----------+-------+--------------------+------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+-------+--------+--------+--------+----------+------------+-------+---------------+----------------+---------------+----------------+-----------------+----------------+
|unique_key|taxi_id|trip_start_timestamp|trip_end_timestamp|trip_seconds|trip_miles|pickup_census_tract|dropoff_census_tract|pickup_community_area|dropoff_community_area|   fare|    tips|   tolls|  extras|trip_total|payment_type|company|pickup_latitude|pickup_longitude|pickup_location|dropoff_latitude|dropoff_longitude|dropoff_location|
+----------+-------+--------------------+------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+-------+--------+--------+--------+----------+------------+-------+---------------+----------------+---------------+----------------+-----------------+-

In [ ]:
null_counts.toPandas()